In [1]:
import os, random, shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
from PIL import Image 

print(tf.__version__)


1.15.3


In [11]:
tf.compat.v1.estimator.experimental.KMeans(
    num_clusters, model_dir=None, initial_clusters=RANDOM_INIT,
    distance_metric=SQUARED_EUCLIDEAN_DISTANCE, seed=None, use_mini_batch=True,
    mini_batch_steps_per_iteration=1, kmeans_plus_plus_num_retries=2,
    relative_tolerance=None, config=None, feature_columns=None
)

NameError: name 'RANDOM_INIT' is not defined

In [2]:

num_points = 100
dimensions = 2
points = np.random.uniform(0, 1000, [num_points, dimensions])

def input_fn():
  return tf.train.limit_epochs(
      tf.convert_to_tensor(points, dtype=tf.float32), num_epochs=1)

num_clusters = 5
kmeans = tf.estimator.experimental.KMeans(
    num_clusters=num_clusters, use_mini_batch=False)

print(points)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbmubm89c', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d61d07748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
[[377.38910473 104.74534292]
 [579.02512306 912.07747595

In [6]:
# train
num_iterations = 10
previous_centers = None
for _ in range(num_iterations):
    kmeans.train(input_fn)
    cluster_centers = kmeans.cluster_centers()
    if previous_centers is not None:
        print('delta:', cluster_centers - previous_centers)
    previous_centers = cluster_centers
    print('score:', kmeans.score(input_fn))
print('cluster centers:', cluster_centers)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
Prefer Dataset.range instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpbmubm89c/model.ckpt.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpbmubm89c/model.ckpt.
INFO:tensorflow:Loss for final step: None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calli

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-14T14:27:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbmubm89c/model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2020-12-14-14:27:02
INFO:tensorflow:Saving dict for global step 9: global_step = 9, loss = 3644198.0, score = 3644198.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9: /tmp/tmpbmubm89c/model.ckpt-9
score: 3644198.0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbmubm89c/model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9 into /tmp/tmpbmubm

In [7]:
# map the input points to their clusters
cluster_indices = list(kmeans.predict_cluster_index(input_fn))
for i, point in enumerate(points):
    cluster_index = cluster_indices[i]
    center = cluster_centers[cluster_index]
    print('point:', point, 'is in cluster', cluster_index, 'centered at', center)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbmubm89c/model.ckpt-19
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
point: [377.38910473 104.74534292] is in cluster 3 centered at [490.95142 144.20364]
point: [579.02512306 912.07747595] is in cluster 2 centered at [790.75 841.53]
point: [423.03040561 962.06251057] is in cluster 0 centered at [363.2021  774.69073]
point: [128.94694852 620.43790778] is in cluster 4 centered at [114.00517 419.15088]
point: [532.3783333  152.35488011] is in cluster 3 centered at [490.95142 144.20364]
point: [992.3242847  917.13719476] is in cluster 2 centered at [790.75 841.53]
point: [  6.07519354 413.15902586] is in cluster 4 centered at [114.00517 419.15088]
point: [301.87975672 706.21852363] is in cluster 0 centered at [363.2021  774.69073]
point: [ 45.27784618 554.78610491] is in cluster 4 centered at [11

In [8]:
kmeans.cluster_centers()

array([[363.2021 , 774.69073],
       [763.88403, 372.14542],
       [790.75   , 841.53   ],
       [490.95142, 144.20364],
       [114.00517, 419.15088]], dtype=float32)

In [9]:
tf.train.load_variable(kmeans.model_dir, KMeansClustering.CLUSTER_CENTERS_VAR_NAME)

NameError: name 'KMeansClustering' is not defined

In [10]:
experimental_export_all_saved_models(
    export_dir_base, input_receiver_fn_map, assets_extra=None, as_text=False,
    checkpoint_path=None
)

NameError: name 'experimental_export_all_saved_models' is not defined

In [13]:
def serving():
    inputs = {}
   # for feat in df.columns:
   #     inputs[feat] = tf.placeholder(shape=[None], dtype = tf.float32)
    inputs = tf.placeholder(shape=[None,9], dtype = tf.float32)
    return tf.estimator.export.TensorServingInputReceiver(inputs,inputs)

In [15]:
kmeans.export_saved_model("./data", 
                          serving_input_receiver_fn=serving,
                          checkpoint_path='./data',
                          experimental_mode=tf.estimator.ModeKeys.PREDICT)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['all_distances', 'cluster_index', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ./data


DataLossError: 2 root error(s) found.
  (0) Data loss: Unable to open table file ./data: Failed precondition: data; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
	 [[node save/RestoreV2 (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
	 [[save/RestoreV2/_1]]
  (1) Data loss: Unable to open table file ./data: Failed precondition: data; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
	 [[node save/RestoreV2 (defined at /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/hom